In [1]:
import gym 
import slimevolleygym

env = gym.make('SlimeVolley-v0')
env.reset()
env.action_space.sample()

array([0, 0, 1], dtype=int8)

In [2]:
from slimevolleygym import BaselinePolicy
policy = BaselinePolicy()
policy.predict(env.reset())


[1, 1, 0]

In [3]:
from domain.config import games
from domain.task_gym import GymTask

game = games["slimevolley"]
task = GymTask(game)

In [4]:
from neat_src import loadHyp, updateHyp
from domain import load_task

hyp_default = 'p/default_neat.json'
hyp_adjust = "p/volley.json"
fileName = "volley"

hyp = loadHyp(pFileName=hyp_default, load_task=load_task)
updateHyp(hyp,load_task,hyp_adjust)

	*** Running with hyperparameters:  p/volley.json 	***


In [5]:
from neat_src import DataGatherer, Neat 

neat = Neat(hyp)

pop = neat.ask()

indiv = pop[0]

task.testInd(indiv.wVec, indiv.aVec) # works now

-5